In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import wordnet_ic
from nltk.tokenize import word_tokenize

# Importamos el algoritmo de lesk
# para desambiguación
from nltk.wsd import lesk

# Importamos las variantes mejoradas
# del algoritmo de Lesk
from pywsd import simple_lesk
from pywsd import adapted_lesk

- **Ejercicio 1:** Utilizando WordNet, obtén los diferentes sentidos de la palabra “fight”. Para cada
uno de ellos imprime su definición, los diferentes lemas asociados al mismo sentido
y, también, ejemplos de uso de cada sentido. 

In [ ]:
# Ej. 1, tema 2 listado V
for s in wordnet.synsets("fight"):
    print(f"Def: {s.definition()}")
    lemmas = []
    for l in s.lemmas():
        lemmas.append(l.name())
    print(lemmas)
    print(s.examples())
    print()

---
- **Ejercicio 2:** Haz lo mismo que en el ejercicio anterior, pero obteniendo los synsets solo cuando la categoría de la palabra sea un verbo. ¿Qué ha cambiado?

In [ ]:
# Ej. 2, tema 2 listado V
for s in wordnet.synsets("fight"):
    if s.pos() == wordnet.VERB:
        print(f"Def: {s.definition()}")
        lemmas = []
        for l in s.lemmas():
            lemmas.append(l.name())
        print(lemmas)
        print(s.examples())
        print()

---
- **Ejercicio 3:** Obtener todos los sentidos de “bank” cuando es un nombre. ¿Cuántos sentidos
tiene? Buscar relaciones entre ellos:
    - Buscar los hiperónimos
    - Buscar los hipónimos  
Un hipónimo concreta el significado de su hiperónimo, de esta manera “mesa” es más específico que “mueble” y “escritorio” es un tipo particular de “mesa”.

In [ ]:
# Ejercicio 3, tema 2 listado V
num_sentidos = 0
for s in wordnet.synsets("bank"):
    if s.pos() == wordnet.NOUN:
        num_sentidos += 1
        hypernyms = []
        hyponyms = []
        for h in s.hypernyms():
            hypernyms.append(h.name())
        print(f"Hiperónimos: {hypernyms}")
        for h in s.hyponyms():
            hyponyms.append(h.name())
        print(f"Hipónimos: {hyponyms}")
        print()
print(f"Número de sentidos de bank siendo un nombre: {num_sentidos}")

---
- **Ejercicio 4:** Se puede calcular la similitud semántica entre dos synsets. WordNet tiene varias medidas de similitud semántica disponibles:
    - Medidas basadas en path, donde se mide la longitud que hay en el camino que lleva de un concepto a otro y cuánto más corto sea el camino, más similares serán (`path_similarity`).
    - Medidas basadas en el contenido, de forma que cuanta más información compartan dos conceptos, más similares serán (`res_similarity`).
    - Medidas basadas en características comunes que pueden compartir los conceptos, y cuántas más características comunes tengan, más similares son (en NLTK no hay).
    - Medidas híbridas, que combinan todas las anteriores (`lch_similarity`).

In [ ]:
# Ejercicio 4, tema 2 listado V
dog = wordnet.synset("dog.n.01")
cat = wordnet.synset("cat.n.01")
vehicle = wordnet.synset("vehicle.n.01")
car = wordnet.synset("car.n.01")
whale = wordnet.synset("whale.n.01")
eagle = wordnet.synset("eagle.n.01")

In [ ]:
def get_similiarities(synset1, synset2, ic):
    return {
        f"Path similarity ({synset1}-{synset2})": synset1.path_similarity(synset2),
        f"Resnik similarity ({synset1}-{synset2})": synset1.res_similarity(synset2, ic),
        f"Leacock-Chodorow similarity ({synset1}-{synset2})": synset1.lch_similarity(synset2, ic)
    }

def print_similarities(sim):
    for similarity, value in sim.items():
        print(f"{similarity} -> {value}")

brown_ic = wordnet_ic.ic('ic-brown.dat')

In [ ]:
# dog - cat
sim_dog_cat = get_similiarities(dog, cat, brown_ic)
print_similarities(sim_dog_cat)

In [ ]:
# vehicle - car
sim_vehicle_car = get_similiarities(vehicle, car, brown_ic)
print_similarities(sim_vehicle_car)

In [ ]:
# dog - car
sim_dog_car = get_similiarities(dog, car, brown_ic)
print_similarities(sim_dog_car)

In [ ]:
# whale - eagle
sim_whale_eagle = get_similiarities(whale, eagle, brown_ic)
print_similarities(sim_whale_eagle)

---
- **Ejercicio 5:** Dada la palabra "watch", buscar los synsets en WordNet y mostrar la definición y las frases de ejemplo. ¿Cómo identificar de qué categoría gramatical es cada synset obtenido?

In [ ]:
for synset in wordnet.synsets("watch"):
    print(f"Synset: {synset.name()}")
    print(f"Definición: {wordnet.synset(synset.name()).definition()}")
    print(f"Frases de ejemplo: {wordnet.synset(synset.name()).examples()}")
    print(f"Categ. gramatical: {wordnet.synset(synset.name()).pos()}")
    print()

- **Ejercicio 6:** Se quiere hacer desambiguación del sentido de una palabra. Una estrategia muy simple (y poco precisa) para desambiguar palabras consiste en seleccionar un synset de WordNet al azar sin tener en cuenta el contexto en el que se encuentra la palabra a analizar. Aunque esta estrategia no es de mucha utilidad en la práctica, permite comprender la importancia de utilizar métodos más sofisticados. Se pide desambiguar la palabra “watch” en la frase “You must watch this film”, seleccionando para ello el primer synset identificado por WordNet. Muestra la definición del synset. ¿Es la acepción correcta de la palabra?

In [ ]:
print(wordnet.synset(wordnet.synsets("watch")[0].name()).definition())

Observamos que no corresponde al sentido de la palabra que se utiliza en la frase "You must watch this film", ya que el primer synset de "watch" en WordNet corresponde al NOUN que describe un reloj portátil.

---
- **Ejercicio 7:** En muchas ocasiones, el significado de una palabra polisémica puede determinarse a partir de su categoría gramatical. Por ejemplo, la palabra “watch” adquiere distintas acepciones dependiendo de si funciona como verbo o sustantivo. En este ejercicio, se pide desambiguar la palabra “watch” en función de la categoría gramatical que presenta en dos frases diferentes. Específicamente se pide:
    - Identificar la categoría gramatical de la palabra “watch” dentro de las frases “You must watch this film” y “He gave me a watch as a present”.
    - A partir de la categoría gramatical, seleccionar el primer synset obtenido en WordNet.
    - Observar el resto de synsets que Wordnet proporciona al especificar la categoría gramatical. ¿Ha mejorado la búsqueda de la acepción correcta? ¿Es suficiente?


In [ ]:
frase1 = "You must watch this film"
frase2 = "He gave me a watch as a present"
word_tokenized1 = word_tokenize(frase1)
word_tokenized2 = word_tokenize(frase2)
tags1 = nltk.pos_tag(word_tokenized1)
tags2 = nltk.pos_tag(word_tokenized2)

In [ ]:
print(tags1)
print(tags2)

In [ ]:
print(wordnet.synset("watch.v.01").definition())
print(wordnet.synset("watch.n.01").definition())

In [ ]:
watch_v_synsets = wordnet.synsets("watch", pos=wordnet.VERB)
watch_n_synsets = wordnet.synsets("watch", pos=wordnet.NOUN)

print("Synsets cuando watch es un verbo")
for s in watch_v_synsets:
    print(f"Synset: {s.name()}")
    print(f"Definición: {s.definition()}")
print()
print("Synsets cuando watch es un nombre")
for s in watch_n_synsets:
    print(f"Synset: {s.name()}")
    print(f"Definición: {s.definition()}")

Podemos observar que la búsqueda de la acepción correcta ha mejorado, pero no es suficiente, ya que dependemos del orden en que están ordenadas las acepciones de una palabra en WordNet y no siempre se va a utilizar la primera de ellas.

---
- **Ejercicio 8:** Lo más razonable a la hora de desambiguar el sentido de las palabras es tener en cuenta el contexto en el que se encuentran. Uno de los primeros algoritmos desarrollados para abordar este problema es el algoritmo de Lesk. Dada una palabra polisémica y el contexto en el que aparece, el algoritmo de Lesk devuelve el synset con el mayor número de palabras coincidentes entre la frase en la que se encuentra la palabra a desambiguar y las distintas definiciones de los synsets que puede poseer una palabra. 
    - Se pide usar el algoritmo de Lesk para desambiguar la palabra “bank” en las frases “I went to the bank to deposit my money” y “The land along the river bank has vegetation”. Muestra la definición del synset. ¿Son las acepciones correctas de la palabra? **Nota**: https://www.nltk.org/howto/wsd.html
    - Con el paso del tiempo se han ido proponiendo mejoras en el algoritmo original desarrollado por Lesk. Algunas de estas mejoras no solo consideran las definiciones de los synsets proporcionados por WordNet, sino también sus hiperónimos, hipónimos y frases de ejemplo, proporcionando un mayor contexto con el que comparar. Se pide Utiliza algoritmos derivados de Lesk para, nuevamente, desambiguar la palabra “bank” en las mismas frases del apartado anterior. Muestra la definición del synset. ¿Son las acepciones correctas de la palabra? **Nota**: Usar los métodos simple_lesk() / adapted_lesk() de la librería pywsd (https://github.com/alvations/pywsd).

In [ ]:
# EJ. 8, a) Algoritmo de Lesk
frase1 = "I went to the bank to deposit my money"
frase2 = "The land along the river bank has vegetation"
word_tokenized1 = word_tokenize(frase1)
word_tokenized2 = word_tokenize(frase2)
synset1 = lesk(word_tokenized1, "bank")
synset2 = lesk(word_tokenized2, "bank")
print(f"Definición de bank en la primera frase: {synset1.definition()}")
print(f"Definición de bank en la segunda frase: {synset2.definition()}")

Utilizando el algoritmo de Lesk obtenemos una acepción incorrecta de la palabra "bank" en la primera frase, ya que el sentido devuelto es el de una hucha en lugar de una entidad que ofrece la posibilidad a sus clientes de depositar y operar con su capital. En cambio, en la segunda frase si se devuelve la acepción correcta del término.

In [ ]:
# EJ. 8, b) Mejoras del algoritmo de Lesk
synset1_simple = simple_lesk(frase1, "bank")
synset1_adapted = adapted_lesk(frase1, "bank")
synset2_simple = simple_lesk(frase2, "bank")
synset2_adapted = adapted_lesk(frase2, "bank")
print(f"Definición de bank en la primera frase (Simple Lesk): {synset1_simple.definition()}")
print(f"Definición de bank en la primera frase (Adapted Lesk): {synset1_adapted.definition()}")
print(f"Definición de bank en la segunda frase (Simple Lesk): {synset2_simple.definition()}")
print(f"Definición de bank en la segunda frase (Adapted Lesk): {synset2_adapted.definition()}")

Utilizando `simple_lesk()` y `adapted_lesk()` obtenemos la acepción correcta del término "bank" en ambos casos.